<a href="https://colab.research.google.com/github/anoy1729/Data_mining/blob/main/DM_unigram_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install python-docx
!pip install git+https://github.com/banglakit/bengali-stemmer.git

     |████████████████████████████████| 5.5MB 9.7MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=323db9d6d736ff5321da3a46da90ce8e0dff08439fa88ea0e5b15b3c705a5cb6
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx
  Cloning https://github.com/banglakit/bengali-stemmer.git to /tmp/pip-req-build-dfspy6w1
  Running command git clone -q https://github.com/banglakit/bengali-stemmer.git /tmp/pip-req-build-dfspy6w1
  Created wheel for bengali-stemmer: filename=bengali_stemmer-0.0.1-py2.py3-none-any.whl size=6393 sha256=c6b5711e18ce7048acccdf73d3927caed22a95e06b27e4beb9fe3708eb1566e6
  Stored in directory: /tmp/pip-ephem-wheel-cache-5xgfdm77/wheels/a1/ad/a1/4ba354059b17c00600a14e13a504e7bdb49f20f2f4e2f3639c
Successfully built bengali-stemmer


In [ ]:
import docx
from bengali_stemmer.rafikamal2014 import RafiStemmer
import string
import pandas as pd
import gensim
import csv

def read_doc_as_pandasDF(filename):

    data = pd.read_csv(filename, error_bad_lines=False)
    data_text = data[['content']]

    data_text['index'] = data_text.index
    documents = data_text

    return(documents)
def punctuation_remover(text):
    BENGALI_PUNCTUATION = string.punctuation + "—।’‘"
    BENGALI_NUMERALS = "০১২৩৪৫৬৭৮৯"
    return text.translate(str.maketrans(' ', ' ', BENGALI_PUNCTUATION+BENGALI_NUMERALS))
def load_stop_word(doc_dir = r"/content/drive/My Drive/Dm_Bigram/stopword-dictionary.docx"):

    stop_directory = doc_dir

    doc = docx.Document(stop_directory)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)

    bengali_stop_words = fullText[0].split()
    bengali_stop_words = frozenset(bengali_stop_words)

    return bengali_stop_words
def preprocess_documents(doc):

    preprocessed_list_of_docs = []
    stemmer = RafiStemmer()

    stop_words = load_stop_word()
    preprocessed_docs = []

    doc_token = []

    if isinstance(doc, str):
        for token in punctuation_remover(doc).split():
            if token not in stop_words and len(token) >= 3:
                if len(stemmer.stem_word(token)) >= 2:
                    doc_token.append(stemmer.stem_word(token))


    return doc_token
def prepare_bag_of_words(processed_docs, dictionary):
    return [dictionary.doc2bow(doc) for doc in processed_docs]
def prepare_bow_list(bow_corpus, dictionary):

    header_list = list(range(0, len(dictionary)-1))
    all_list = [header_list]

    for each_list in bow_corpus:
        temp_list = [0]*len(dictionary)
        for each_tuple in each_list:
            temp_list[each_tuple[0]] = each_tuple[1]
        all_list.append(temp_list)

    minimal_all_list = []

    minimal_header_list = []

    for i in range(len(dictionary)):
        minimal_header_list.append(dictionary[i])

    minimal_all_list.append(minimal_header_list)

    for each_mini_list in all_list[1:]:
        minimal_all_list.append(each_mini_list)


    return(minimal_all_list)   
def process_nonstemmed_documents(doc):
    preprocessed_list_of_docs = []

    stop_words = load_stop_word()
    preprocessed_docs = []

    doc_token = []

    if isinstance(doc, str):
        for token in punctuation_remover(doc).split():
            if token not in stop_words and len(token) >= 3:
                doc_token.append((token))


    return doc_token

def get_bow_list(bow_corpus, dictionary):

    full_list  = [['token', 'count']]
    token_list = [0]*len(dictionary)


    for each_doc in bow_corpus:
        for each_tuple in each_doc:
            token_list[each_tuple[0]] += each_tuple[1]

    for i, token_count in enumerate(token_list):
        full_list.append([dictionary[i], token_count])

    return(full_list)

if __name__ == "__main__":
    CSV_LOCATION = r"/content/drive/My Drive/Dm_Bigram/amadershomoy.csv"
    
    pd_document = read_doc_as_pandasDF(CSV_LOCATION)

    smaller_documents = pd_document[0:]
    #processed_docs = smaller_documents['content'].map(preprocess_documents)
    processed_docs = smaller_documents['content'].map(process_nonstemmed_documents)


    dictionary = gensim.corpora.Dictionary(processed_docs)
    #dictionary.filter_extremes(no_below=.01, no_above=0.6, keep_n=100000)

    bow_corpus = prepare_bag_of_words(processed_docs, dictionary)

    #print(bow_corpus)

    minimal_all_list = get_bow_list(bow_corpus, dictionary)

    
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [ ]:
with open("/content/drive/My Drive/Dm_Bigram/sorted_uni.csv", "w", newline="", encoding="utf-8") as f:
        minimal_all_list = sorted(minimal_all_list)
        print(minimal_all_list)
        writer = csv.writer(f)
        writer.writerows(minimal_all_list)
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

